In [ ]:
pip install -q yahoo_fin

In [2]:
!mkdir -p data results logs


In [3]:
from google.colab import drive
drive.mount('/content/drive')
!cp  /content/drive/MyDrive/colab/results/* ./results/

Mounted at /content/drive


In [ ]:
pip install  git+https://github.com/ahsank/runml@testv2.01#egg=runml

In [5]:
import runml.pipeline
from runml import pipeline,findata


In [6]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    #df['Gain_f'] = (2*df['Gain']+df['Gain_l']+df['Gain_h'])/4
    df['Gain_f'] = df[['Gain','Gain_l']].max(axis=1)
    df['Gain_f'] = df[['Gain_h', 'Gain_f']].min(axis=1)
    df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain_f'])
    df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
    return df

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
new = []
model = "gstock-10b-test"
tickers = [ 'AEXAF', 'ALSMY', 'AMSSY', 'ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY', 'CAAP', 'COTY', 'COVTY', 'CDMGF', 'CPG', 'CX',
            'DBOEY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY', 'EQNR', 'ERIC', 'ERJ','ESEA',
            'FANUY', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'OZON', 'RACE', 'LYB', 'LVMUY',
            'MYTE', 'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'VWAPY', 'YARIY', 'ZURVY']


In [ ]:
findata.EPOCHS=200
model = "val-10b-test"
new = ['CCI', 'TMUS', 'CLS', 'POWL', 'ANF', 'MOD', 'STRL',  'MPC', 'BXC',
       'URI', 'DINO', 'TEX', 'POR', 'HIW']
tickers = ['AAL', 'ALB', 'ANF', 'APO', 'AQN', 'ARCH', 'ARE',
            'BAH', 'BAX', 'BGFV', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CHK', 'CI', 'CIVI',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'EMN',
            'F', 'FCX', 'FIS', 'FLO', 'FMS', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IP', 'IPI', 'IVZ', 'JWN', 'KD', 'KHC', 'KMB',
             'M', 'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL', 'PXD', 'RCL', 'RH',
            'SAFE', 'SCCO', 'SPTN', 'SPG', 'STRL', 'SWK',
            'T', 'TCN', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']

In [ ]:
findata.EPOCHS=200
model = "cry-11a-test"
tickers = [ 'ADA-USD', 'BTC-USD', 'DOT-USD', 'DOGE-USD',
            'ETH-USD', 'LTC-USD', 'MATIC-USD', 'XRP-USD']

In [10]:
findata.EPOCHS=200
model = "etf-11a-test"
tickers = ['ARKF', 'ARKK', 'ARKW', 'DAPP', 'DIA', 'DTEC', 'EEM', 'FPX',
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'VNQ', 'VT', 'VTI', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]

In [ ]:
findata.EPOCHS=200
model = "mcap-9b-test"
tickers = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX', 'TMO', 'TSLA', 'TXN', 'UNH', 'UPS']

In [ ]:
pipeline.IS_VERBOSE = False
lossfn = "huber_loss"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))


df  = pipeline.runModelCombinedVola(tickers, model, mod, False, loss=lossfn)


In [9]:
addAllocHL(df, 0.10, 1)
df.iloc[(df.Alloc * abs(df.Gain_f)).sort_values(ascending=False).index].head(50)
#df.sort_values('Alloc', ascending=False).head(50)


,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Gain_f,Alloc
47,PANW,48.87,0.87,1144.35,681.54,253.51,181.600006,-0.28,0.74,1047.31,380.43,254.190002,0.00,0.80,1423.80,844.05,191.130005,-0.25,-0.25,8.180000
42,NVCR,15.41,0.89,1253.22,1228.54,11.13,15.340000,0.38,0.71,1022.96,1118.05,13.750000,0.24,0.78,794.52,1610.72,13.300000,0.20,0.24,8.441667
52,PI,14.74,0.88,796.25,542.52,71.74,54.660000,-0.24,0.71,661.14,615.78,65.919998,-0.08,0.71,719.59,629.23,48.610001,-0.32,-0.24,8.300000
48,PAYO,3.61,0.89,44.48,46.48,5.44,6.610000,0.22,0.68,38.50,48.80,6.690000,0.23,0.79,42.08,43.25,5.490000,0.01,0.22,8.400000
63,TOST,13.24,0.91,97.06,135.13,14.11,17.120001,0.21,0.73,65.49,114.76,18.520000,0.31,0.68,88.04,131.56,14.930000,0.06,0.21,8.671429
51,PLUG,4.47,0.80,321.02,305.40,3.53,4.410000,0.25,0.64,117.68,146.02,4.860000,0.38,0.70,167.55,261.29,3.610000,0.02,0.25,7.200000
33,JKS,14.31,0.90,509.31,467.75,36.12,29.100000,-0.19,0.68,714.26,676.20,39.500000,0.09,0.78,678.94,827.27,28.700001,-0.21,-0.19,8.473684
4,APP,11.19,0.95,293.70,407.21,43.79,51.349998,0.17,0.70,246.08,262.57,51.400002,0.17,0.76,330.02,444.64,43.830002,0.00,0.17,9.205882
39,MPWR,141.88,0.90,3452.27,2787.11,504.58,400.250000,-0.21,0.74,2911.42,2660.91,499.869995,-0.01,0.78,4608.80,2825.42,411.510010,-0.18,-0.18,8.444444
8,CFLT,19.75,0.72,174.13,281.58,17.68,21.980000,0.24,0.59,140.10,270.93,23.760000,0.34,0.72,69.03,300.33,16.580000,-0.06,0.24,6.033333


In [ ]:
findata.EPOCHS=200
model = "vols-10a-test"
pending = ['ASTR']
new = ['FSR', 'PROK', 'YALA', 'AXTI', 'REI', 'NNOX', 'ACHR', 'GRAB', 'SOUN']
tickers = [ 'AIRI', 'APRN', 'AUGX', 'AUR', 'AMSC',  'ATOM',
            'BKKT', 'BGSF', 'BLNK', 'BSGM',
            'CARS', 'CLOV', 'CZOO',
            'DLO',  'DM', 'DMTK', 'DPSI', 'EVGO', 'EXPR', 'FSLY', 'FTCH',
            'GOVX', 'GTE', 'HIMS', 'HIVE',
            'INLX',  'INVZ', 'JOBY',  'KIND', 'KULR',
            'LAZR', 'LUMN', 'MTTR',
            'NEPH', 'NSTG', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PTON', 'RAD', 'REAL', 'RIG', 'RKLB', 'RVYL',
            'SFIX', 'STEM', 'SFT',  'TELL', 'VTEX', 'YEXT']
future = ['WKME', 'CXM']

In [7]:
pending = ['PAR']
findata.EPOCHS=200
model = "ipos-10a-test"
new = ['SMCI', 'TRIP']
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BILL', 'BMBL', 'CELH', 'CFLT', 'CHGG', 'CRCT', 'CRWD',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DXCM',
           'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FOUR', 'GFS', 'GTLB', 'GLBE', 'GOGO',
           'INDI', 'INMD', 'INTA', 'IOT', 'IRDM', 'JKS', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL',
           'MTCH',  'NVCR','OKTA', 'OLED', 'OSTK',
           'PERI', 'PANW', 'PAYO', 'PAYX', 'PD', 'PLUG', 'PI', 'PINS', 'PUBM',
           'RBLX', 'SNAP', 'SNOW', 'SPLK', 'SQ', 'SPOT',
           'TEAM', 'TDOC', 'TOST', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']

In [ ]:
findata.EPOCHS=200
model = "cetf-9b-test"
tickers = ['ARGT', 'EWD', 'FLN', 'FM',
            'ECH', 'EWA', 'EWI', 'EWC', 'EWM', 'EWP', 'EWW', 'EWY', 'EWZ',
            'IDX', 'ILF', 'KEMQ', 'THD', 'TUR', 'XCEM']

In [ ]:
pip install seaborn

In [ ]:
import seaborn as sns

Todos:



1.   Load previously saved model before training
2.   Allow eval only without test/train spliting
3.   Incremental data load
4.    Add S&P, QQQ etc to the model
5. Save model to google drive






In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  !cp  /content/drive/MyDrive/colab/results/* ./results/
else:
  ! cp ~/Google\ Drive/My\ Drive/colab/results/* ./results/

Run following in local Jupyter

Run following 2 cells for colab to import libraries

Run following if the library code is modified locally

In [ ]:
!git clone https://github.com/ahsank/runml

In [ ]:
import os
import sys
path = os.path.expanduser('~/src/runml')
print(path)
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata

/Users/ahsank/src/runml


In [ ]:
import importlib
import sys
path = 'runml'
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata


In [ ]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

In [ ]:
from importlib import reload
reload(findata)
reload(pipeline)

<module 'runml.pipeline' from '/Users/ahsank/src/runml/runml/pipeline.py'>

In [ ]:
findata.EPOCHS=200
model = "com-11a-test"
tickers = [ 'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'UUP', 'VTIP', 'WEAT']

In [ ]:
lossfn = "huber_loss"
pipeline.IS_VERBOSE = False
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))


#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='low', trading=pipeline.LowTrading)[0]
#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='high', trading=pipeline.HighTrading)[0]
df  = pipeline.runModelCombinedVola(tickers, model, mod, True, loss=lossfn)

In [12]:
addAllocHL(df, 0.10, 1)
df.iloc[(df.Alloc * abs(df.Gain_f)).sort_values(ascending=False).index].head(50)
#df.sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Gain_f,Alloc
3,DAPP,2.97,0.85,85.82,118.82,5.87,9.140000,0.56,0.68,56.71,84.37,8.180000,0.39,0.79,58.76,115.86,6.470000,0.10,0.39,8.115385
1,ARKK,30.84,0.92,504.98,726.71,39.73,47.709999,0.20,0.76,484.32,576.89,47.080002,0.18,0.73,687.10,790.37,43.000000,0.08,0.18,8.755556
2,ARKW,37.92,0.94,756.42,617.14,58.90,71.029999,0.21,0.77,497.04,578.61,68.660004,0.17,0.77,648.42,859.26,60.450001,0.03,0.17,9.047059
0,ARKF,14.14,0.97,254.38,262.73,20.44,24.330000,0.19,0.74,178.05,198.18,23.410000,0.15,0.76,189.44,303.34,20.910000,0.02,0.15,9.500000
21,ONLN,26.96,0.94,331.92,360.32,30.65,35.330002,0.15,0.69,283.44,213.84,34.169998,0.11,0.79,312.62,352.12,29.530001,-0.04,0.11,8.854545
23,SMH,50.95,0.94,698.71,495.48,157.28,144.740005,-0.08,0.78,629.08,577.04,151.529999,-0.04,0.72,717.82,473.36,140.630005,-0.11,-0.08,8.650000
8,ICLN,8.34,0.91,127.04,115.99,13.01,14.000000,0.08,0.81,155.13,107.33,14.150000,0.09,0.71,86.71,96.18,11.950000,-0.08,0.08,7.975000
45,XSW,70.94,0.95,741.17,676.69,127.54,136.139999,0.07,0.78,730.48,559.00,136.240005,0.07,0.77,745.90,753.21,115.949997,-0.09,0.07,8.785714
17,JETS,12.22,0.95,114.72,141.83,15.69,16.850000,0.07,0.80,149.88,162.68,17.540001,0.12,0.76,133.97,166.60,15.320000,-0.02,0.07,8.785714
10,IPO,22.31,0.97,271.17,326.12,31.60,33.619999,0.06,0.82,387.64,250.66,33.880001,0.07,0.74,265.20,365.64,31.389999,-0.01,0.06,9.200000


In [ ]:
import pandas as pd
df1 = pipeline.fetch_data('ABNB')
#df1.sort_index(inplace=True)
df1 = df1.round(3)

df2 = pd.read_csv('data/ABNB_2023-04-15.csv', index_col=0)
df2.index =  pd.to_datetime(df2.index)
# df2 = df2.round(3)
#data2 = data.tail(1000)
#pd.concat([df1,df2]).drop_duplicates(keep=False)
#df1.compare(df2)
#df2.index.dtype
df1

In [ ]:
display(df.sort_values('Gain', ascending=False))
display(df[["Buy", "Sell", "Total"]].sum()/len(df))


In [ ]:
results['ABNB'].pdata.__dict__.keys()

In [ ]:
findata.EPOCHS=10
tickers1 = ['ARKK', 'ARKW', 'DAPP', 'DTEC', 'EEM', 'FPX',
            'GBTC', 'GLD',
            'ICLN', 'IJR', 'IPO', 'IPOS', 'IWM',
            'JETS', 'KEMQ',
            'MGK', 'MGV', 'MOAT', 'MTUM',
            'QQQ', 'SLV','SMH', 'SMOG',
            'SPY', 'VNQ', 'VT', 'VTI', 'WDIV',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME' ]
mod = pipeline.RateReturnOnly(pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap()]))
df1 = pipeline.runModelCombined(tickers1, 'etf2b', mod, True)

In [ ]:
df1.sort_values('Gain', ascending=False)

In [ ]:
findata.EPOCHS=200
tickers3 = [
           'AHT', 'AMSC', 'ASTR', 'ATOM',
           'BKKT', 'BGFV', 'BGSF', 'CBNT', 'CLOV',
           'DNMR', 'ERJ', 'EVGO',
           'FSLY',  'FTCH', 'GOGO', 'HIVE',
           'ILAL', 'INLX', 'JMIA', 'JOBY',  'KULR', 'MTTR',
           'MYTE', 'NEPH', 'ONDS', 'MQ',
           'PETS', 'PTON', 'SFIX', 'SFT', 'STNE',
           'ULH', 'VRAR', 'WISH']
lossfn = "huber_loss"
# lossfn = "mean_squared_error"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df3, results = pipeline.runModelCombined(tickers3, 'vols', mod, False, loss=lossfn)

In [ ]:
df3

In [ ]:
import pandas as pd
seq = list()
for ticker, result in results.items():
    tempdf = result.final_df
    tempdf['predicted_rate'] = (tempdf['adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    tempdf['true_rate'] = (tempdf['true_adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    seq.append(tempdf[['ticker', 'date', "predicted_rate", 'true_rate']])

detailstat = pd.concat(seq)



In [ ]:
normalstat = detailstat[detailstat['predicted_rate'].between(-2,2) & detailstat['true_rate'].between(-2,2)]
largestat = normalstat[~ normalstat['predicted_rate'].between(-0.25,0.25) & ~ normalstat['true_rate'].between(-0.25,0.25)]
largestat.sample(10)

In [ ]:
ax = largestat.plot.scatter(x='predicted_rate', y='true_rate',c='DarkBlue', figsize=(10,10))
# ax.set_yscale('log')
# ax.set_xscale('log')

In [ ]:
# import matplotlib.pyplot as plt

# plt.hist([normalstat['predicted_rate'],normalstat['predicted_rate']],
#          bins=100, range=(-1,1), color = ['r','g'])

corr = largestat[["predicted_rate", "true_rate"]].corr()

corr.style.background_gradient(cmap='coolwarm')

In [ ]:

sns.regplot(x=largestat["predicted_rate"], y=largestat["true_rate"])


In [ ]:
sns.heatmap(normalstat[["predicted_rate", "true_rate"]].corr(), annot = True, fmt='.2g',cmap= 'coolwarm')

In [13]:
!rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/


sending incremental file list
etf-11a-test-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
etf-11a-test-high-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
etf-11a-test-low-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5

sent 9,648,089 bytes  received 73 bytes  19,296,324.00 bytes/sec
total size is 189,533,488  speedup is 19.64
